In [1]:
import pandas as pd

# Notionのデータを読み込む
tag_skill_df = pd.read_csv("./Notion/sample.csv")
tag_skill_df

,TAG-ID,スキル項目コード,知識項目コード
0,TAG-1,S210030050,K013
1,TAG-17,S140030050,K034
2,TAG-17,S140030050,K035
3,TAG-17,S210010070,K060
4,TAG-17,S210020120,K021
5,TAG-17,S220020040,K004
